## Example 4: Pricing in GE with ARIMA Approximation
### Background
Suppose now that there is general equilibirum feedback with the degree of strategic complementarity $\alpha$: $$p_{i,t}^*=(1-\alpha)q_t+\alpha p_t$$ where
\begin{align}
    \Delta q_t&=\rho \Delta q_{t-1}+u_t,\quad u_t\sim \mathcal{N}(0,\sigma_u^2) \\
    p_t&\equiv \int_0^1 p_{i,t}di
\end{align}
Note that now the state space representation for $p_{i,t}^*$ is no longer exogenous and is determined in the equilibrium. However, we know that this is a Guassian process and can be approximated by an ARIMA($p$,$d$,$q$) process. Here, $d$ indicates whether the process has a unit root (which is true in the case of pricing problems). Moreover, we will choose $p$ and $q$ to minimize approximation error subject the maximizing performance. Larger state spaces are more accurate but increase the dimensionality of the DRIP and slow down the algorithm.

In our case $d=1$ and hence $dp_{i,t}^*$ is stationary and approximated by an ARMA($p$,$q$):
$$\Delta p_{i,t}^*=\sum_{j=1}^p \rho_i \Delta p_{i,t-j}^*+\sum_{k=1}^q \theta_i u_{t-k}+\tilde{\sigma} u_t$$

Now, let 
$$\vec{x}_t = (p_{i,t}^*,\Delta p_{i,t}^*,\dots,\Delta p_{i,t-p+1}^*,u_t,\dots,u_{t-q+1})'\in\mathbb{R}^{p+q+1}$$

### Matrix Notation

Given a guess for the vector $\Gamma\equiv (\rho_1,\dots,\rho_p,\theta_1,\dots,\theta_q)$ and a guess for $\tilde{\sigma}$ we have
$$\vec{x}_t=\mathbf{A}(\Gamma)\vec{x}_{t-1}+\mathbf{Q}(\tilde{\sigma})u_t$$ where

\begin{align}
    \mathbf{A}(\Gamma)=\left[\begin{array}{ccccc}
        1 &| &&\Gamma'&\\ \hline
        0 &| &&\Gamma'&\\ \hline
        0 &| \mathbf{I}_{(p-1)\times(p-1)}&|  & \mathbf{0}_{(p-1)\times(q+1)} & \\ \hline
         &  & & \mathbf{0}_{1\times(p+q+1)} & \\ \hline
        0 & \mathbf{0}_{(q-1)\times(p+2)} &| & \mathbf{I}_{(q-1)\times(q-1)} & 
        \end{array}\right], \quad 
    \mathbf{Q}(\hat{\sigma})=\left[\begin{array}{c}
         \tilde{\sigma} \\
         \tilde{\sigma} \\ \hline
         \mathbf{0}_{(p-1)\times 1} \\ \hline
         1 \\ \hline
         \mathbf{0}_{(q-1)\times 1} 
        \end{array}\right]
\end{align}
and 
\begin{align}
    p_{i,t}^*=\mathbf{e}_1'\vec{x}_t, \quad \mathbf{e}_1'=(1,0,\dots,0) \in \mathbb{R}^{1\times (p+q+1)}
\end{align}

Our objective is to find the equilibrium $(\Gamma,\tilde{\sigma})$, and we do this by iteration for an initial guess $(\Gamma_{(0)},\tilde{\sigma}_{(0)})$:

* At iteration $n\geq 0$, using function solve the ratinoal inattention problem given the guess $(\Gamma_{(n-1)},\tilde{\sigma}_{(n-1)})$.

* Get the implied impulse response of $\Delta p_{i,t^*}$ and approximate it with an ARMA(p,q).

* Update $(\Gamma_{(n)},\tilde{\sigma}_{(n)})$.
* Iterate until convergence.

### Setup the problem:

In [106]:
## Model Parameters
ρ   = 0.6;        #persistence of money growth
σ_u = .1;          #std. deviation of shocks to money growth
α   = 0.8;        #degree of strategic complementarity
ω   = .4;          #cost of attention
β   = 0.99;       #discount factor

## Approximation Parameters
p   = 2;          #p in ARMA(p,q)
q   = 2;          #q in ARMA(p,q)
T   = 60;         #length if irfs
Hq  = ρ.^(0:T-1); #irf of Δq

Γ0,σ̃0 = arma_approx(Hq,p,q)
A = [[1;Γ0]';
                 [0;Γ0]';
                 [zeros(1,p-1);Matrix(I,p-1,p-1);zeros(q+1,p-1)]';
                  zeros(1,p+q+1);
                 [zeros(p+2,q-1);Matrix(I,q-1,q-1)]'];
Q = [σ̃0;σ̃0;zeros(p-1,1);1;zeros(q-1,1)];
inv(A*A'+Q*Q')

5×5 Array{Float64,2}:
  1.0           -1.0        -0.0           -0.0        -0.0        
 -1.0           11.6697     -3.56279      -10.6697     -8.399e-16  
  0.0           -3.56279     2.18968        3.56279     2.80457e-16
 -5.55112e-16  -10.6697      3.56279       11.6697      8.399e-16  
 -1.84889e-32   -8.399e-16   2.80457e-16    8.399e-16   1.0        

In [107]:
function arma_approx(H,p,q; T=length(H), M = [zeros(1,T-1) 0; Matrix(I,T-1,T-1) zeros(T-1,1)])
    B = zeros(p+q+1,T);
    for i in 1:p
        B[q+1+i,:] = H'*M'^i;
    end
    B[1:q+1,1:q+1] = Matrix(I,q+1,q+1);
    Γ̄ = inv(B*B')*B*H;
    σ̃ = Γ̄[1];
    Γ = [Γ̄[q+2:q+p+1];Γ̄[2:q+1]];
    return(Γ,σ̃)
end

function ge_drip(ω,β,                     #primitives of drip 
                 α,                       #strategic complementarity
                 p,q,                     #parameters of ARMA(p,q)
                 Hq;                      #irf of Δq
                 maxit = 200,             #optional: max number of iterations for GE code 
                 tol   = 1e-12)           #optional: tolerance for iterations

    # set primitives
    Γ0,σ̃0 = arma_approx(Hq,p,q);
    T     = length(Hq);
    err   = 1;  
    iter  = 0;
    M     = [zeros(1,T-1) 0; Matrix(I,T-1,T-1) zeros(T-1,1)];
    eye   = Matrix(I,T,T);
    H     = zeros(p+q+1,1); H[1]=1;
    
    # iterate on GE
    while (err > tol) & (iter < maxit)
            A = [[1;Γ0]';
                 [0;Γ0]';
                 [zeros(1,p-1);Matrix(I,p-1,p-1);zeros(q+1,p-1)]';
                  zeros(1,p+q+1);
                 [zeros(p+2,q-1);Matrix(I,q-1,q-1)]'];
            Q = [σ̃0;σ̃0;zeros(p-1,1);1;zeros(q-1,1)];
            if iter == 0 
                global ge  = Solve_RI_Dynamics(ω,β,A,Q,H); 
            else 
                global ge  = Solve_RI_Dynamics(ω,β,A,Q,H;Ω0 = ge.Ω ,Σ0 = ge.Σ_1);
            end
            K = real.(ge.K)
            Y = real.(ge.Y)

             if maximum(abs.(imag.(ge.K))) > 1e-6 || maximum(abs.(imag.(ge.Y))) > 1e-6
                 println("Complex components are large")
             end

            irfs  = dripirfs(ge,T);
            Hdp   = (1-α)*Hq+α*(I-M)*irfs.a[1,1,:];
            Γ1,σ̃1 = arma_approx(Hdp,p,q);
            err   = norm([Γ1;σ̃1]-[Γ0;σ̃0],2);
        
            Γ0,σ̃0 = Γ1,σ̃1;
        
            if iter == maxit
                print("GE loop hit maxit -- no convergence\n")
            end
            iter += 1;
            println("Iteration $iter. Difference: $err")
    end
    return(ge)
end     

ge_drip (generic function with 3 methods)